In [1]:
import time
import hmac
import requests
import arrow
import pandas as pd
import asyncio
from datetime import datetime as dt
from pprint import pprint
from tqdm import tqdm

In [2]:
#Tidying the DataFrame
def DForginaser(dfname,newname):
    newname = dfname.drop('startTime',axis=1)
    newname['Date'] = [arrow.Arrow.fromtimestamp(date).format('YYYY-MM-DD HH:mm') for date in newname['time']]
    newname.drop('time',axis=1,inplace=True)
    colname = {
                'Date':'Date',
                'open':'Open',
                'high':'High',
                'low':'Low',
                'close':'Close',
                'volume':'Volume'
                }
    newname.rename(columns=colname, inplace=True)
    col_order = ['Date','Open','High','Low','Close','Volume']
    newname = newname[col_order]
    newname.sort_values(by='Date',inplace=True,ascending=True)
    return newname

In [5]:
#window length in seconds. options: 15, 60, 300, 900, 3600, 14400, 86400, or any multiple of 86400 up to 30*86400
#This function will create a csv file with containing the historical prices of the specified tickers 
def GetHistoFTX(tickers,startdate,enddate,timeframe):
    #date format as arg must be YYYY-MM-DD
    resolution = timeframe 
    for ticker in tickers:
        end_ = arrow.get(enddate).timestamp
        datelimit = arrow.get(startdate).timestamp - resolution #creates a loop limit
        nbline = 1000
        start_ = end_ - (resolution*nbline)
        ticker_df = []
        while start_ > datelimit:
            #print(dt.fromtimestamp(end_),dt.fromtimestamp(start_))
            url = f'https://ftx.com/api/markets/{ticker}/candles'.format()
            params_histo = dict(resolution = resolution,
                    limit =1500,
                    start_time=int(start_),
                    end_time=int(end_))
            r = requests.get(url=url,params=params_histo)
            data = r.json()
            time.sleep(0.1)

            if end_ == data['result'][0]['time']/1000:
                break
            elif data['success'] is 'False':        
                break
            else:
                end_ = round(data['result'][0]['time']/1000,0)

            for i in data['result']:
                ticker_df.append(i)
            
            start_ = end_ - (resolution*nbline)

        ticker_df = pd.DataFrame(ticker_df)
        filename = [ticker.replace('/','-') if '/' in ticker else ticker][0]
        ticker_df = DForginaser(ticker_df,filename)
        path=r'your path'
        ticker_df.to_csv(f'{path}\{filename}.csv'.format(),index=False)

In [6]:
#below example will create 4 csv files as per the 4 tickers specified in the list tickers
#Within a year period from 20 Dec 2020 to 20 Dec 2021 on a hourly timeframe (3600 resolution)
tickers = ['ETH/USD','BTC/USD','ETH-PERP','FTM-PERP']
GetHistoFTX(tickers,'2020-12-20','2021-12-20',3600)